In [30]:
import numpy as np
from numpy import linalg as LA
import math

In [27]:
x=np.array([[0,1],[2,3]])

In [38]:
X = np.array([[0, 2, 2], [0, 2, 1], [0, 2, 0],[1, 2, 0],[2, 2, 0],[0, 1, 2],[0, 1, 1],[0, 1, 0],[1, 1, 0],[2, 1, 0], [0, 0, 2],[0, 0, 1],[0, 0, 0],[1, 0, 0],[2, 0, 0]])

In [28]:
def normalizing_params(pts):
    centroid=np.mean(pts,axis=0)
    avg_distance=np.mean(LA.norm(pts-centroid, axis=1))
    return centroid,avg_distance


In [34]:
def normalizing_matrix(pts):
    c,d=normalizing_params(pts)
    n=pts.shape[1]
    d=d/math.sqrt(2)
    if n==2:
        T=np.array([[1/d,0,-c[0]/d],[0,1/d,-c[1]/d],[0,0,1]])
    else:
        T=np.array([[1/d,0,0,-c[0]/d],[0,1/d,0,-c[1]/d],[0,0,1/d,-c[2]/d],[0,0,0,1]])
    return T

In [39]:
normalizing_params(X)

(array([0.6, 1. , 0.6]), 1.3453274243577624)

In [40]:
normalizing_matrix(X)

array([[ 1.05120399,  0.        ,  0.        , -0.6307224 ],
       [ 0.        ,  1.05120399,  0.        , -1.05120399],
       [ 0.        ,  0.        ,  1.05120399, -0.6307224 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])